In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
       # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import joblib
x = joblib.load("/kaggle/input/preprocessing-done/x_data.joblib")
y = joblib.load("/kaggle/input/preprocessing-done/y_data.joblib")
img_ids = joblib.load("/kaggle/input/preprocessing-done/img_ids.joblib")
print(x.shape)
print(y.shape)

(10763, 256, 256, 3)
(10763,)


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
x1=x[:8000]
y1=y[:8000]
img_ids1=img_ids[:8000]
x_train, x_test, y_train, y_test, img_ids_train, img_ids_test = train_test_split(x1, y1, img_ids1, test_size=0.15, shuffle=True, random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


(6800, 256, 256, 3)
(1200, 256, 256, 3)
(6800,)
(1200,)


In [4]:
import tensorflow as tf
print(tf.test.is_gpu_available())

True


a two-layer network (CNN) with one convolutional layer (four feature maps, kernel width 120, stride 1) with ReLU activation, one fully connected layer, softmax activation, and L1-regularization with weight 0.0001 was applied.  

# CNN

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import save_model
devices = ['/GPU:0', '/GPU:1']
num_classes = 6
strategy = tf.distribute.MirroredStrategy(devices=devices)
# Build the CNN model
with strategy.scope(): 
    CNN_model= Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    CNN_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    CNN_model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test,y_test))  # 训练模型
    save_model(CNN_model, "/kaggle/working/CNN_model.h5")
    predictions=CNN_model.predict(x_test)
    y_pred = np.argmax(predictions, axis=1)

Epoch 1/3
107/107 [==============================] - 28s 138ms/step - loss: 2.2788 - accuracy: 0.2743 - val_loss: 1.5323 - val_accuracy: 0.3442
Epoch 2/3
107/107 [==============================] - 10s 90ms/step - loss: 1.5120 - accuracy: 0.3374 - val_loss: 1.5097 - val_accuracy: 0.2925
Epoch 3/3
107/107 [==============================] - 10s 93ms/step - loss: 1.4089 - accuracy: 0.4157 - val_loss: 1.5324 - val_accuracy: 0.3308


/tmp/ipykernel_26/408422091.py:24: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(CNN_model, "/kaggle/working/CNN_model.h5")


38/38 [==============================] - 1s 20ms/step


In [6]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.3308333333333333

In [7]:
from collections import Counter

def find_majority_label(image_ids, labels):
    id_label_map = {}  # 创建一个空的字典来存储image_id和对应的label

    # 遍历image_ids和labels列表
    for image_id, label in zip(image_ids, labels):
        if image_id in id_label_map:
            id_label_map[image_id].append(label)
        else:
            id_label_map[image_id] = [label]

    majority_labels = {}  # 创建一个空的字典来存储每个唯一image_id的多数投票label

    # 对字典中的每个唯一image_id的label列表进行多数投票
    for image_id, label_list in id_label_map.items():
        label_counts = Counter(label_list)
        majority_label = label_counts.most_common(1)[0][0]
        majority_labels[image_id] = majority_label

    return majority_labels

y_votingpred=find_majority_label(img_ids_test, y_pred)
# 对图像 ID 进行去重
unique_img_ids = np.unique(img_ids_test)

# 根据去重后的图像 ID，获取对应的真实标签和预测标签
y_test_matched = []
y_test_matched = np.array([y_test[np.where(img_ids_test == img_id)[0][0]] for img_id in unique_img_ids])
y_votingpred_matched = np.array([y_votingpred[img_id] for img_id in unique_img_ids])

# 计算准确度
accuracy = accuracy_score(y_test_matched, y_votingpred_matched)
accuracy 

0.3723404255319149